# **Train Data Preprocessing Notebook**

This notebook processes train data collected from https://trainstats.altervista.org/.

The goal is to load, clean, and structure the dataset to make it suitable for further analysis.

The dataset contains information about train schedules, delays, and stops.

In [1]:
import os
import json
import pandas as pd
import numpy as np
from tqdm import tqdm
from pathlib import Path
from datetime import datetime

In [2]:
BASE_PATH = Path("data/raw/trains_dataset")
OUTPUT_PATH = Path("data/interim")
OUTPUT_PATH.mkdir(parents=True, exist_ok=True)

**Loading JSON Data**

We loop through each month and each daily JSON file, extracting:
- General statistics (total monitored trains, delays, etc.).
- Detailed train information (delays per train).
- Alerts from RFI/TI about disruptions.

Finally, all datasets are concatenated into full-year DataFrames.

In [3]:
def load_all_data(base_path=BASE_PATH):
    all_data = []
    
    # Scan monthly folders (1_2024, 2_2024, ...)
    for month_folder in sorted(base_path.iterdir()):
        if month_folder.is_dir():
            print(f" Processing {month_folder.name}...")
            
            for file_path in tqdm(sorted(month_folder.glob("*.json"))):
                try:
                    with open(file_path, "r", encoding="utf-8") as f:
                        data = json.load(f)
                        
                        if "treni" in data:
                            all_data.extend(data["treni"])
                except (json.JSONDecodeError, FileNotFoundError) as e:
                    print(f" Errore nel file {file_path}: {e}")
                    continue

    return pd.DataFrame(all_data)

In [4]:
df = load_all_data()

 Processing 10_2024...


100%|██████████| 31/31 [00:03<00:00,  8.33it/s]


 Processing 11_2024...


100%|██████████| 30/30 [00:03<00:00,  8.87it/s]


 Processing 12_2024...


...

100%|██████████| 31/31 [00:05<00:00,  6.02it/s]


 Processing 8_2024...


100%|██████████| 31/31 [00:02<00:00, 14.76it/s]


 Processing 9_2024...


100%|██████████| 30/30 [00:05<00:00,  5.13it/s]


**Renaming Columns**

The original column names are abbreviated. We'll map short codes to more descriptive column names.

In [5]:
column_mapping = {
    "_id": "train_id",
    "n": "train_number",
    "p": "departure_station",
    "rp": "train_departure_delay",
    "a": "arrival_station",
    "ra": "train_arrival_delay",
    "dl": "delay_info",
    "c": "train_class",
    "oo": "origin_station",
    "od": "final_destination",
    "op": "scheduled_departure_time",
    "oa": "scheduled_arrival_time",
    "pr": "train_status",  # (Soppresso = Canceled)
    "sub": "train_subclass",
    "sea": "extended_final_destination",
    "cn": "connected_train",
    "oae": "official_scheduled_arrival",
    "oaz": "adjusted_scheduled_arrival",
    "opz": "adjusted_scheduled_departure",
    "ope": "official_planned_departure",
    "sep": "starting_extended_point",
    "fr": "route_stops"
}

df.rename(columns=column_mapping, inplace=True)

**Converting Time Columns**

Convert UNIX timestamps into datetime format and ensure valid date ranges.

In [6]:
def safe_to_datetime(series):
    """Convert timestamps to datetime safely, ensuring valid ranges."""
    MIN_TIMESTAMP = 1703980800  # Dec 31, 2023
    MAX_TIMESTAMP = 1735756800  # Jan 1, 2025

    if pd.api.types.is_datetime64_any_dtype(series):
        return series  

    series = pd.to_numeric(series, errors="coerce")

    series = series.where((series >= MIN_TIMESTAMP) & (series <= MAX_TIMESTAMP))

    return pd.to_datetime(series, unit="s", errors="coerce")

time_columns = ["scheduled_departure_time", "scheduled_arrival_time",
                "adjusted_scheduled_arrival", "adjusted_scheduled_departure"]

for col in time_columns:
    df[col] = safe_to_datetime(df[col])

In [7]:
df["train_departure_delay"] = pd.to_numeric(df["train_departure_delay"], errors="coerce")
df["train_arrival_delay"] = pd.to_numeric(df["train_arrival_delay"], errors="coerce")

In [8]:
df.head()

,train_id,train_number,departure_station,train_departure_delay,arrival_station,train_arrival_delay,train_status,train_class,scheduled_departure_time,scheduled_arrival_time,...,origin_station,final_destination,connected_train,train_subclass,adjusted_scheduled_arrival,extended_final_destination,official_scheduled_arrival,starting_extended_point,official_planned_departure,adjusted_scheduled_departure
0,10911-1727734500-TUlMQU5PIENFTlRSQUxF,10911,MILANO CENTRALE,NaN,BRESCIA,NaN,Soppresso,REG,2024-09-30 22:15:00,2024-09-30 23:35:00,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
1,17431-1727735880-Qk9MT0dOQSBDLkxF,17431,BOLOGNA C.LE,1.0,FAENZA,-1.0,NaN,REG,2024-09-30 22:38:00,2024-09-30 23:25:00,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
2,18650-1727735100-RklSRU5aRSBTLk0uTi4=,18650,FIRENZE S.M.N.,1.0,PISTOIA,1.0,NaN,REG,2024-09-30 22:25:00,2024-09-30 23:13:00,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
3,5636-1727735100-UFVOVEEgUkFJU0k=,5636,PUNTA RAISI,1.0,PALERMO CENTRALE,-1.0,NaN,REG,2024-09-30 22:25:00,2024-09-30 23:05:00,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT
4,17099-1727734140-VkVORVpJQSBTQU5UQSBMVUNJQQ==,17099,VENEZIA SANTA LUCIA,3.0,PORTOGRUARO CAORLE,2.0,NaN,REG,2024-09-30 22:09:00,2024-09-30 23:23:00,...,NaN,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,NaT


**Handling Train Stops**

The "route_stops" column contains a list of dictionaries with stop details. We normalize them into a separate DataFrame.

In [9]:
df_exploded = df.explode("route_stops").reset_index(drop=True)

# Remove any NaN values in "route_stops"
df_exploded = df_exploded[df_exploded["route_stops"].notna()]

# Normalize the stops
df_stops = pd.json_normalize(df_exploded["route_stops"])

stop_column_mapping = {
    "n": "stop_name",
    "ra": "stop_arrival_delay",
    "rp": "stop_departure_delay",
    "br": "actual_platform",
    "bp": "planned_platform",
    "oa": "stop_arrival_time",
    "op": "stop_departure_time"
}

df_stops.rename(columns=stop_column_mapping, inplace=True)

df_stops["train_id"] = df_exploded["train_id"].values
df_stops["train_number"] = df_exploded["train_number"].values

time_columns = ["stop_arrival_time", "stop_departure_time"]
for col in time_columns:
    df_stops[col] = safe_to_datetime(df_stops[col])

In [10]:
df_stops.head()

,stop_name,stop_arrival_delay,stop_departure_delay,stop_arrival_time,stop_departure_time,actual_platform,planned_platform,train_id,train_number
0,MILANO CENTRALE,N,S,NaT,2024-09-30 22:15:00,NaN,NaN,10911-1727734500-TUlMQU5PIENFTlRSQUxF,10911
1,MILANO LAMBRATE,S,S,2024-09-30 22:21:00,2024-09-30 22:22:00,NaN,NaN,10911-1727734500-TUlMQU5PIENFTlRSQUxF,10911
2,PIOLTELLO LIMITO,S,S,2024-09-30 22:29:00,2024-09-30 22:30:00,NaN,NaN,10911-1727734500-TUlMQU5PIENFTlRSQUxF,10911
3,MELZO,S,S,2024-09-30 22:35:00,2024-09-30 22:36:00,NaN,NaN,10911-1727734500-TUlMQU5PIENFTlRSQUxF,10911
4,CASSANO D`ADDA,S,S,2024-09-30 22:42:00,2024-09-30 22:43:00,NaN,NaN,10911-1727734500-TUlMQU5PIENFTlRSQUxF,10911


**Merging Dataframes**

Merge the train and stops datasets into a single DataFrame.

In [11]:
df_merged = df.merge(df_stops, on="train_id", how="left")

In [12]:
df_merged.head()

,train_id,train_number_x,departure_station,train_departure_delay,arrival_station,train_arrival_delay,train_status,train_class,scheduled_departure_time,scheduled_arrival_time,...,official_planned_departure,adjusted_scheduled_departure,stop_name,stop_arrival_delay,stop_departure_delay,stop_arrival_time,stop_departure_time,actual_platform,planned_platform,train_number_y
0,10911-1727734500-TUlMQU5PIENFTlRSQUxF,10911,MILANO CENTRALE,NaN,BRESCIA,NaN,Soppresso,REG,2024-09-30 22:15:00,2024-09-30 23:35:00,...,NaN,NaT,MILANO CENTRALE,N,S,NaT,2024-09-30 22:15:00,NaN,NaN,10911
1,10911-1727734500-TUlMQU5PIENFTlRSQUxF,10911,MILANO CENTRALE,NaN,BRESCIA,NaN,Soppresso,REG,2024-09-30 22:15:00,2024-09-30 23:35:00,...,NaN,NaT,MILANO LAMBRATE,S,S,2024-09-30 22:21:00,2024-09-30 22:22:00,NaN,NaN,10911
2,10911-1727734500-TUlMQU5PIENFTlRSQUxF,10911,MILANO CENTRALE,NaN,BRESCIA,NaN,Soppresso,REG,2024-09-30 22:15:00,2024-09-30 23:35:00,...,NaN,NaT,PIOLTELLO LIMITO,S,S,2024-09-30 22:29:00,2024-09-30 22:30:00,NaN,NaN,10911
3,10911-1727734500-TUlMQU5PIENFTlRSQUxF,10911,MILANO CENTRALE,NaN,BRESCIA,NaN,Soppresso,REG,2024-09-30 22:15:00,2024-09-30 23:35:00,...,NaN,NaT,MELZO,S,S,2024-09-30 22:35:00,2024-09-30 22:36:00,NaN,NaN,10911
4,10911-1727734500-TUlMQU5PIENFTlRSQUxF,10911,MILANO CENTRALE,NaN,BRESCIA,NaN,Soppresso,REG,2024-09-30 22:15:00,2024-09-30 23:35:00,...,NaN,NaT,CASSANO D`ADDA,S,S,2024-09-30 22:42:00,2024-09-30 22:43:00,NaN,NaN,10911


In [13]:
df_merged.drop(columns=["route_stops"], inplace=True)
df_merged.drop(columns=["train_number_y"], inplace=True)
df_merged.rename(columns={"train_number_x": "train_number"}, inplace=True)

## **Cleaning Missing Values and Removing Redundant Columns**

We analyze missing values and remove non-informative or redundant columns.

In [14]:
# Calculation and visualization of missing values
missing_values = df_merged.isnull().sum()
missing_percentage = (missing_values / len(df_merged)) * 100

missing_summary = pd.DataFrame({
    "Missing Values": missing_values,
    "Percentage": missing_percentage.round(2)
})

missing_summary = missing_summary[missing_summary["Missing Values"] > 0]
print("\n Null values in the final dataset:")
print(missing_summary)



 Null values in the final dataset:
                              Missing Values  Percentage
train_departure_delay                 270676        0.82
train_arrival_delay                   270672        0.82
train_status                        32728594       99.18
delay_info                          32355738       98.05
origin_station                      32351228       98.04
final_destination                   32351228       98.04
connected_train                     32573766       98.71
train_subclass                      32164517       97.47
adjusted_scheduled_arrival          32989216       99.97
extended_final_destination          32988668       99.97
official_scheduled_arrival          32988780       99.97
starting_extended_point             32987350       99.96
official_planned_departure          32987350       99.96
adjusted_scheduled_departure        32992976       99.98
stop_name                                115        0.00
stop_arrival_delay                       115        

In [15]:
# Number of rows before cleaning
initial_rows_df = df_merged.shape[0]

**We remove rows containing suppressed trains**

In [16]:
# Remove rows where train_status is "Soppresso"
df_merged = df_merged[df_merged["train_status"] != "Soppresso"]

In [17]:
# Remove columns with too many missing values or non-informative ones
columns_to_drop = [
    "train_status",
    "delay_info",  # Textual, not useful for prediction
    "origin_station", "final_destination",  # Few valid and redundant values
    "connected_train", "train_subclass", "adjusted_scheduled_arrival",
    "extended_final_destination", "official_scheduled_arrival",
    "starting_extended_point", "official_planned_departure", "adjusted_scheduled_departure",
    "actual_platform", "planned_platform"  # Removal of columns with >99% missing values
]
df_merged.drop(columns=columns_to_drop, inplace=True)

In [18]:
# Calculation and visualization of missing values
missing_values = df_merged.isnull().sum()
missing_percentage = (missing_values / len(df_merged)) * 100

missing_summary = pd.DataFrame({
    "Missing Values": missing_values,
    "Percentage": missing_percentage.round(2)
})

missing_summary = missing_summary[missing_summary["Missing Values"] > 0]
print("\n Null values in the final dataset:")
print(missing_summary)



 Null values in the final dataset:
                       Missing Values  Percentage
train_departure_delay               4        0.00
stop_arrival_time             2781911        8.50
stop_departure_time           2804887        8.57


**We remove deleted stops**

The problem is that some scheduled stops have been cancelled, so the train did not make the stop at those stations. 

In the example, we see that train 2855 was supposed to arrive at Milano Rogoredo, but it was cancelled on that route and ends its journey at Milano Centrale. 

We solve by removing those lines.

<div style="max-height: 300px; overflow-y: auto; border: 1px solid #ccc; padding: 10px;" 
     onmouseover="this.style.overflowY='auto'" 
     onmouseout="this.style.overflowY='hidden'">

```json
 {
      "_id": "2855-1704086040-Q09MSUNP",
      "n": "2855",
      "p": "COLICO",
      "rp": "1",
      "a": "MILANO CENTRALE",
      "ra": "-3",
      "dl": "Treno cancellato da SESTO S. GIOVANNI a MILANO ROGOREDO. Il treno oggi arriva a MILANO CENTRALE.",
      "c": "REG",
      "oo": "COLICO",
      "od": "MILANO ROGOREDO",
      "cn": "73467,MILANO GRECO PIRELLI",
      "op": 1704086040,
      "oa": 1704093120,
      "fr": [
        {
          "n": "COLICO",
          "ra": "N",
          "rp": "1",
          "oa": 0,
          "op": 1704086040
        },
        {
          "n": "PIONA",
          "ra": "-2",
          "rp": "1",
          "oa": 1704086340,
          "op": 1704086400
        },
        {
          "n": "DORIO",
          "ra": "0",
          "rp": "-1",
          "oa": 1704086700,
          "op": 1704086760
        },
        {
          "n": "DERVIO",
          "ra": "-3",
          "rp": "0",
          "oa": 1704087060,
          "op": 1704087120
        },
        {
          "n": "BELLANO TARTAVELLE TERME",
          "ra": "-1",
          "rp": "5",
          "oa": 1704087480,
          "op": 1704087540
        },
        {
          "n": "VARENNA ESINO",
          "ra": "4",
          "rp": "5",
          "oa": 1704087780,
          "op": 1704087840
        },
        {
          "n": "FIUMELATTE",
          "ra": "4",
          "rp": "4",
          "oa": 1704087990,
          "op": 1704088020
        },
        {
          "n": "LIERNA",
          "ra": "3",
          "rp": "4",
          "oa": 1704088320,
          "op": 1704088380
        },
        {
          "n": "OLCIO",
          "ra": "3",
          "rp": "3",
          "oa": 1704088590,
          "op": 1704088620
        },
        {
          "n": "MANDELLO DEL LARIO",
          "ra": "1",
          "rp": "2",
          "oa": 1704088860,
          "op": 1704088920
        },
        {
          "n": "ABBADIA LARIANA",
          "ra": "1",
          "rp": "6",
          "oa": 1704089160,
          "op": 1704089400
        },
        {
          "n": "LECCO",
          "ra": "5",
          "rp": "4",
          "oa": 1704089820,
          "op": 1704090000
        },
        {
          "n": "CALOLZIOCORTE OLGINATE",
          "ra": "2",
          "rp": "3",
          "oa": 1704090420,
          "op": 1704090480
        },
        {
          "n": "CERNUSCO-MERATE",
          "ra": "1",
          "rp": "1",
          "oa": 1704091080,
          "op": 1704091140
        },
        {
          "n": "CARNATE USMATE",
          "ra": "-3",
          "rp": "-1",
          "oa": 1704091560,
          "op": 1704091620
        },
        {
          "n": "MONZA",
          "ra": "-2",
          "rp": "1",
          "oa": 1704092160,
          "op": 1704092220
        },
        {
          "n": "SESTO S. GIOVANNI",
          "ra": "-1",
          "rp": "1",
          "oa": 1704092520,
          "op": 1704092580
        },
        {
          "n": "MILANO LAMBRATE",
          "ra": "S",
          "rp": "S",
          "oa": 1704093240,
          "op": 1704093300
        },
        {
          "n": "MILANO FORLANINI",
          "ra": "S",
          "rp": "S",
          "oa": 1704093540,
          "op": 1704093600
        },
        {
          "n": "MILANO ROGOREDO",
          "ra": "S",
          "rp": "S",
          "oa": 1704093960,
          "op": 0
        },
        {
          "n": "MILANO CENTRALE",
          "ra": "-3",
          "rp": "N",
          "oa": 1704093120,
          "op": 0
        }
      ]
    }

In [19]:
# Remove stops with "S" in arrival or departure delay
df_merged = df_merged[~df_merged["stop_arrival_delay"].astype(str).str.upper().eq("S")]
df_merged = df_merged[~df_merged["stop_departure_delay"].astype(str).str.upper().eq("S")]

print(f"Skipped stops removed. The final dataset contains {len(df_merged)} stops.")


Skipped stops removed. The final dataset contains 32549240 stops.


**Schedule not available with NaT**.

Stops that have ra = “n.d.” (arrival delay not available) or rp = “n.d.” (departure delay unavailable) represent a special case.
In the example, the Serravalle Scrivia stop has ra = “n.d.,” which means that the arrival delay for that station is not known.

We solve replacacing “n.d.” with NaN (np.nan). 
This allows machine learning models to handle the value as missing without skewing the data.

<div style="max-height: 300px; overflow-y: auto; border: 1px solid #ccc; padding: 10px;">

```json
{
      "_id": "2116-1704086820-R0VOT1ZBIFAuUFJJTkNJUEU=",
      "n": "2116",
      "p": "GENOVA P.PRINCIPE",
      "rp": "2",
      "a": "TORINO P.NUOVA",
      "ra": "-3",
      "c": "REG",
      "op": 1704086820,
      "oa": 1704094200,
      "fr": [
        {
          "n": "GENOVA P.PRINCIPE",
          "ra": "N",
          "rp": "2",
          "oa": 0,
          "op": 1704086820
        },
        {
          "n": "RONCO SCR.",
          "ra": "0",
          "rp": "2",
          "oa": 1704088200,
          "op": 1704088260
        },
        {
          "n": "ARQUATA SCRIVIA",
          "ra": "1",
          "rp": "4",
          "oa": 1704088740,
          "op": 1704088800
        },
        {
          "n": "SERRAVALLE SCRIVIA",
          "ra": "n.d.",
          "rp": "1",
          "oa": 1704089160,
          "op": 1704089220
        },
        ...
      ]
    }

In [20]:
# Convert "n.d." delays to NaN to handle them as missing values
df_merged["stop_arrival_delay"] = df_merged["stop_arrival_delay"].replace("n.d.", np.nan)
df_merged["stop_departure_delay"] = df_merged["stop_departure_delay"].replace("n.d.", np.nan)

df_merged["stop_arrival_delay"] = pd.to_numeric(df_merged["stop_arrival_delay"], errors="coerce")
df_merged["stop_departure_delay"] = pd.to_numeric(df_merged["stop_departure_delay"], errors="coerce")

print(f"Stops with 'n.d.' handled correctly. The final dataset contains {len(df_merged)} stops.")

Stops with 'n.d.' handled correctly. The final dataset contains 32549240 stops.


**Manage terminal station times**

The destination stations do not have a departure time, and similarly, the departure stations do not have an arrival time.

In the json files such times are denoted by '0'. 

To get around this problem, we can leave the missing values as NaT (Not a Time): In pandas dataframes, NaT is the standard for indicating a missing timestamp, just like NaN for numbers. Advanced machine learning models can handle NaT without problems, while an incorrect value such as “0” could compromise predictions.

In addition, we add an is_terminal_stop column: This Boolean (True/False) column indicates whether the stop is the start or end station of the route. In this way, we can easily identify stops where arrival/departure values are missing legitimately.

In [21]:
# Creation of the boolean column for terminal stops
df_merged["is_terminal_stop"] = df_merged["stop_arrival_time"].isna() | df_merged["stop_departure_time"].isna()

# Fill with NaT instead of 0
df_merged["stop_arrival_time"] = pd.to_datetime(df_merged["stop_arrival_time"], errors="coerce")
df_merged["stop_departure_time"] = pd.to_datetime(df_merged["stop_departure_time"], errors="coerce")

In [22]:
true_percentage = (df_merged["is_terminal_stop"].sum() / len(df_merged)) * 100
print(f"Terminal stops percentage: {true_percentage:.2f}%")

Terminal stops percentage: 17.05%


In [23]:
missing_values = df_merged.isnull().sum()
missing_percentage = (missing_values / len(df_merged)) * 100

missing_summary = pd.DataFrame({
    "Missing Values": missing_values,
    "Percentage": missing_percentage.round(2)
})

missing_summary = missing_summary[missing_summary["Missing Values"] > 0]
print("\n Null values in the final dataset:")
print(missing_summary)


 Null values in the final dataset:
                       Missing Values  Percentage
train_departure_delay               4        0.00
stop_arrival_delay            7147420       21.96
stop_departure_delay          3993550       12.27
stop_arrival_time             2765755        8.50
stop_departure_time           2783814        8.55


The sum of missing values in these two columns (8.5% + 8.55% = 17.05%) matches exactly with the percentage of `True` values in `is_terminal_stop`, confirming that our logic for flagging terminal stops is correct.  

If the percentage of `True` in `is_terminal_stop` was higher or lower than the combined missing percentages in `stop_arrival_time` and `stop_departure_time`, it would indicate an inconsistency in the logic used to define terminal stops.  

Since they match, we can be confident that `is_terminal_stop` is a **reliable feature** and accurately reflects whether a stop is terminal.  

**Handling delays at terminal stops**

In the json files, `stop_arrival_delay` and `stop_departure_delay` times are denoted by 'N'. Infact, a train starts its journey from a departure station, meaning there is no previous stop to accumulate delay. A train ends its journey at the final station, meaning it does not have a next stop where departure delay would matter.

Therefore, such null values are not true missing values, but expected structural absences. Filling them with 0 maintains logical consistency, prevents misinterpretation in downstream processing, and ensures clean data for analysis and modeling.

In [24]:
df_merged.loc[df_merged["stop_arrival_time"].isna(), "stop_arrival_delay"] = 0
df_merged.loc[df_merged["stop_departure_time"].isna(), "stop_departure_delay"] = 0

In [25]:
missing_values = df_merged.isnull().sum()
missing_percentage = (missing_values / len(df_merged)) * 100

missing_summary = pd.DataFrame({
    "Missing Values": missing_values,
    "Percentage": missing_percentage.round(2)
})

missing_summary = missing_summary[missing_summary["Missing Values"] > 0]
print("\n Null values in the final dataset:")
print(missing_summary)


 Null values in the final dataset:
                       Missing Values  Percentage
train_departure_delay               4        0.00
stop_arrival_delay            4383167       13.47
stop_departure_delay          1212496        3.73
stop_arrival_time             2765755        8.50
stop_departure_time           2783814        8.55


**Handling Anomalous Delay Values in Train Data**  

During data inspection, an issue was identified with the recorded arrival and departure delays for certain stops. As in the example, in the case of train 17633 at the Argenta station, the recorded arrival delay was an extremely large negative value, which is clearly incorrect.  

A significantly negative delay is unrealistic. Such a massive negative value is likely due to a logging or data entry mistake. Infact, the timestamp for the arrival time (`oa = 100374615540`) is completely out of scale, suggesting that the data point is corrupted or misformatted.  
Such anomalies could distort statistical analyses, predictions, and machine learning models if left unaddressed.  

To ensure data reliability and prevent these errors from affecting further analysis, a data cleaning step was applied to remove rows with clearly anomalous delay values.  

We defined reasonable thresholds for delays. Any records that exceeded these thresholds were automatically filtered out from the dataset.  

<div style="max-height: 300px; overflow-y: auto; border: 1px solid #ccc; padding: 10px;">

```json
{
    "_id": "17633-1715787420-RkVSUkFSQQ==",
    "n": "17633",
    "p": "FERRARA",
    "rp": "4",
    "a": "RAVENNA",
    "ra": "2",
    "c": "REG",
    "op": 1715787420,
    "oa": 1715791680,
    "fr": [
    {
        "n": "FERRARA",
        "ra": "N",
        "rp": "4",
        "oa": 0,
        "op": 1715787420
    },
    {
        "n": "MONTESANTO",
        "ra": "3",
        "rp": "3",
        "oa": 1715788320,
        "op": 1715788380
    },
    {
        "n": "PORTOMAGGIORE",
        "ra": "2",
        "rp": "3",
        "oa": 1715788740,
        "op": 1715788860
    },
    {
        "n": "ARGENTA",
        "ra": "-1644313770",
        "rp": "1",
        "oa": 100374615540,
        "op": 1715789400
    },
    ...
    ]
}
```


In [26]:
# Define reasonable limits for delay in minutes
MIN_DELAY = -10  # Maximum allowed early arrival
MAX_DELAY = 300  # Maximum allowed delay

df_merged = df_merged[
    (df_merged["stop_arrival_delay"].between(MIN_DELAY, MAX_DELAY, inclusive="both")) &
    (df_merged["stop_departure_delay"].between(MIN_DELAY, MAX_DELAY, inclusive="both")) &
    (df_merged["train_departure_delay"].between(MIN_DELAY, MAX_DELAY, inclusive="both")) &
    (df_merged["train_arrival_delay"].between(MIN_DELAY, MAX_DELAY, inclusive="both"))
]

print(f"Anomalous delays removed! The final dataset contains {len(df_stops)} stops.")

Anomalous delays removed! The final dataset contains 32995541 stops.


In [27]:
# Number of rows after cleaning
final_rows_df = df_merged.shape[0]

In [28]:
# Calculation and visualization of missing values
missing_values = df_merged.isnull().sum()
missing_percentage = (missing_values / len(df_merged)) * 100

missing_summary = pd.DataFrame({
    "Missing Values": missing_values,
    "Percentage": missing_percentage.round(2)
})

missing_summary = missing_summary[missing_summary["Missing Values"] > 0]
print("\n Null values in the final dataset:")
print(missing_summary)


 Null values in the final dataset:
                     Missing Values  Percentage
stop_arrival_time           2756872        9.86
stop_departure_time         2774884        9.92


In [29]:
loss_percentage_df = (1 - final_rows_df / initial_rows_df) * 100

print(f"Data lost in the train dataset: {initial_rows_df - final_rows_df} rows ({loss_percentage_df:.2f}%)")

Data lost in the train dataset: 5025841 rows (15.23%)


In [30]:
df_merged.shape

(27973425, 15)

In [31]:
df_merged.describe()

,train_departure_delay,train_arrival_delay,scheduled_departure_time,scheduled_arrival_time,stop_arrival_delay,stop_departure_delay,stop_arrival_time,stop_departure_time
count,2.797342e+07,2.797342e+07,27973425,27973425,2.797342e+07,2.797342e+07,25216553,25198541
mean,2.640689e+00,2.445902e+00,2024-06-29 19:12:30.748817152,2024-06-29 21:01:46.929380096,2.907570e+00,3.994690e+00,2024-06-29 20:09:50.362128384,2024-06-29 20:04:28.495725568
min,-9.000000e+00,-1.000000e+01,2023-12-31 23:02:00,2023-12-31 23:28:00,-1.000000e+01,-1.000000e+01,2023-12-31 23:13:00,2023-12-31 23:02:00
25%,1.000000e+00,-1.000000e+00,2024-03-29 06:05:00,2024-03-29 07:51:00,0.000000e+00,1.000000e+00,2024-03-29 06:38:00,2024-03-29 06:31:00
50%,1.000000e+00,1.000000e+00,2024-06-27 05:20:00,2024-06-27 07:10:00,1.000000e+00,2.000000e+00,2024-06-27 05:56:00,2024-06-27 05:50:00
75%,3.000000e+00,3.000000e+00,2024-10-01 05:31:00,2024-10-01 07:20:00,4.000000e+00,5.000000e+00,2024-10-01 07:18:00,2024-10-01 07:11:00
max,2.890000e+02,3.000000e+02,2024-12-31 22:55:00,2025-01-01 13:40:00,3.000000e+02,2.990000e+02,2025-01-01 13:40:00,2025-01-01 13:34:00
std,4.677534e+00,9.453813e+00,NaN,NaN,7.457632e+00,7.174272e+00,NaN,NaN


In [34]:
df_merged.head(5)

,train_id,train_number,departure_station,train_departure_delay,arrival_station,train_arrival_delay,train_class,scheduled_departure_time,scheduled_arrival_time,stop_name,stop_arrival_delay,stop_departure_delay,stop_arrival_time,stop_departure_time,is_terminal_stop
13,17431-1727735880-Qk9MT0dOQSBDLkxF,17431,BOLOGNA C.LE,1.0,FAENZA,-1.0,REG,2024-09-30 22:38:00,2024-09-30 23:25:00,BOLOGNA C.LE,0.0,1.0,NaT,2024-09-30 22:38:00,True
14,17431-1727735880-Qk9MT0dOQSBDLkxF,17431,BOLOGNA C.LE,1.0,FAENZA,-1.0,REG,2024-09-30 22:38:00,2024-09-30 23:25:00,S.LAZZARO DI SAVENA,2.0,2.0,2024-09-30 22:45:00,2024-09-30 22:46:00,False
15,17431-1727735880-Qk9MT0dOQSBDLkxF,17431,BOLOGNA C.LE,1.0,FAENZA,-1.0,REG,2024-09-30 22:38:00,2024-09-30 23:25:00,OZZANO DELL'EMILIA,3.0,2.0,2024-09-30 22:51:00,2024-09-30 22:52:00,False
16,17431-1727735880-Qk9MT0dOQSBDLkxF,17431,BOLOGNA C.LE,1.0,FAENZA,-1.0,REG,2024-09-30 22:38:00,2024-09-30 23:25:00,CASTEL S.PIETRO TERME,3.0,4.0,2024-09-30 22:58:00,2024-09-30 22:59:00,False
17,17431-1727735880-Qk9MT0dOQSBDLkxF,17431,BOLOGNA C.LE,1.0,FAENZA,-1.0,REG,2024-09-30 22:38:00,2024-09-30 23:25:00,IMOLA,3.0,4.0,2024-09-30 23:07:00,2024-09-30 23:08:00,False


In [35]:
df_merged.columns

Index(['train_id', 'train_number', 'departure_station',
       'train_departure_delay', 'arrival_station', 'train_arrival_delay',
       'train_class', 'scheduled_departure_time', 'scheduled_arrival_time',
       'stop_name', 'stop_arrival_delay', 'stop_departure_delay',
       'stop_arrival_time', 'stop_departure_time', 'is_terminal_stop'],
      dtype='object')

Finally, we need to drop redundant, non-informative, or problematic columns to:  
- Avoid **data leakage** (features not available at prediction time).  
- Reduce **dimensionality** (improve model efficiency).  
- Remove **correlations** that may cause multicollinearity issues.  

Columns to drop:
- **train_id**: Unique identifiers, not useful for learning. 
- **train_arrival_delay**, **stop_arrival_time**: These are post-event values, trhan they are not available in real-world predictions.
- **train_class**: Textual and unbalanced, not useful for prediction

**stop_arrival_delay** will be our target (y)

In [36]:
columns_to_drop = [
    "train_id", 
    "stop_arrival_time",
    "train_arrival_delay", 
    "train_class"
]

df_merged.drop(columns=columns_to_drop, inplace=True)

In [37]:
missing_values = df_merged.isnull().sum()
missing_percentage = (missing_values / len(df_merged)) * 100

missing_summary = pd.DataFrame({
    "Missing Values": missing_values,
    "Percentage": missing_percentage.round(2)
})

missing_summary = missing_summary[missing_summary["Missing Values"] > 0]
print("\n Null values in the final dataset:")
print(missing_summary)



 Null values in the final dataset:
                     Missing Values  Percentage
stop_departure_time         2774884        9.92


In [38]:
INTERIM_PATH = Path("data/interim")
INTERIM_PATH.mkdir(parents=True, exist_ok=True)

df_merged.to_parquet(INTERIM_PATH / "train_data_cleaned.parquet", index=False)

print("Cleaned datasets successfully saved in 'data/interim'")


Cleaned datasets successfully saved in 'data/interim'
